In [1]:
import json
import time
import pandas as pd
from py2neo import Graph, Node, Relationship
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import glob


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
graph = Graph("bolt://neo4j-magtwo:7687", auth=('neo4j','myneo'))
top_42 = ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']

print("Connected to graph database with {:,} nodes and {:,} relationships!".format(
    graph.database.primitive_counts['NumberOfNodeIdsInUse'], 
    graph.database.primitive_counts['NumberOfRelationshipIdsInUse']))

Connected to graph database with 439,314,635 nodes and 563,712,791 relationships!


# Load and Build Dataset from Disk

In [17]:

def read_build_dataset(datapath, years_to_use, target_year):
    df = pd.read_pickle(datapath)
    years_to_use = years_to_use

    df_new = df[['year','title','id']]
    for i in range(years_to_use+1):
        df_new['c{}'.format(i)] = df.apply(lambda row: row['c{}'.format(row['year']+i)], axis=1)
        df_new['p{}'.format(i)] = df.apply(lambda row: row['tspr{}'.format(row['year']+i)], axis=1)

    # ADD TARGET

    target_year = target_year

    df_new['target_pr{}'.format(target_year)] = df.apply(lambda row: row['tspr{}'.format(row['year']+target_year)], axis=1)



    feature_paths = ["/tmp/data/result/FeatureExtractionResults/EarlyAdopters/"]

    for feature_path in feature_paths:
        all_files = sorted(glob.glob(feature_path +"*.csv"), reverse=True)
        feature_vec_chunks = []
        for file in all_files:
            feature_vec_chunks.append(pd.read_csv(file))

        total_feature = pd.concat(feature_vec_chunks)
        df_features = df_new.merge(pd.concat(feature_vec_chunks), on='title')

    df_features = df_features.dropna()
    return df_features

DATAPATH = "/tmp/data/result/datasets/top_42_2000_2015.pkl"
YEARS_TO_USE = 3
TARGET_YEAR = 3
df_features = read_build_dataset(DATAPATH, YEARS_TO_USE, TARGET_YEAR)
titles = df_features['title']
df_features = df_features.drop(columns=['title'])

# Target Creation
y = df_features['target_pr{}'.format(TARGET_YEAR)]
log_y = y.apply(lambda x: np.log(x))







/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

# Signal Selection + Training/Test Split

In [ ]:
from sklearn.model_selection import train_test_split

NUM_YEARS = 1
def signal_select(df_features, num_years):
    to_keep = []
    num_years = num_years
    i = 0
    while True:

        if i == num_years:
            break
        to_keep.append('c{}'.format(i))
        to_keep.append('p{}'.format(i))
        i+=1
    
    i = 1

    while True:
        if i-1 == num_years:
            break
        to_keep.append('early_adopters_{}'.format(i))
        i+=1
        
        

    df_cleaned = df_features[to_keep]

    
    return df_cleaned

df_cleaned = signal_select(df_features, NUM_YEARS)
    

X_train, X_test, y_train, y_test = train_test_split(df_cleaned, log_y, test_size=0.33, random_state=42)

X_train, y_train


# Fit Regression Models

In [ ]:
!pip install scikit-learn
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor



scoring = ['r2', 'neg_mean_squared_error']


model = RandomForestRegressor(random_state=0)
cv_results = cross_validate(model, X_train, y_train, scoring=scoring,
                         cv=5, return_train_score=True)

# Predict Top N

In [ ]:

fitted_model = model.fit(X_train, y_train)

prediction_results = fitted_model.predict(df_cleaned)

result_df = df_cleaned.copy(deep=True).reset_index(drop=True)
result_df['titles'] = titles.reset_index(drop=True)
result_df['predictions'] = pd.Series(prediction_results)


result_df = result_df.sort_values('predictions', ascending=False)

TOP_N = 100
# result_df.iloc[:TOP_N].to_csv('')






In [19]:
DATAPATH = "/tmp/data/result/datasets/top_42_2016_2018.pkl"
YEARS_TO_USE = 1
TARGET_YEAR = 1
df_features_new = read_build_dataset(DATAPATH, YEARS_TO_USE, TARGET_YEAR)
titles_new = df_features_new['title']
df_features_new = df_features_new.drop(columns=['title'])

df_cleaned_new = signal_select(df_features_new, YEARS_TO_USE)
df_cleaned_new

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

2
hi


,c0,p0,early_adopters_1
2508,2.0,0.150000,20
2509,2.0,0.150000,0
2510,3.0,0.277500,0
2511,1.0,0.150000,14
2512,7.0,0.192500,0
2513,3.0,0.532500,0
2514,11.0,0.150000,17
2515,1.0,0.150000,0
2516,3.0,0.150000,3
2517,6.0,0.245625,0


# Data Visualization
## Target Pagerank distribution

In [ ]:
import seaborn as sns

In [ ]:
log_y = y.apply(lambda x: np.log(x))
sns.distplot(log_y)


In [ ]:
sns.distplot(df_cleaned.p0.apply(lambda x: np.log(x) if x > 0 else -15))
sns.distplot(df_cleaned.c0.apply(lambda x: np.log(x) if x > 0 else -15))
sns.distplot(df_cleaned.early_adopters_1.apply(lambda x: np.log(x) if x > 0 else -15))

In [ ]:
titles